In [2]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Копирование файловых дескрипторов и неименованные каналы

<br>
<div style="text-align: right"> 
Спасибо <a href="https://github.com/SyrnikRebirth">Сове Глебу</a> за участие в написании текста     
<br>И я приглашаю всех желающих улучшать материалы
<br>  в этом репозитории, как это можно сделать
<br>  написано в главном README.md 
    
</div>
<br>
Да, dup и pipe мы уже рассматривали в sem13. Так что по этой части будет некоторое повторение (но с большим количеством подробностей).

Сегодня будем рассматривать вызовы:
* `dup`, `dup2`, `dup3` - позволяют "скопировать" файловый дескриптор. То есть получить еще один файловый дескриптор на тот же файл/соединение. Например, так можно скопировать дескриптор файла в 1 файловый дескриптор (stdout) и потом с помощью функции printf писать в этот файл.
  * `dup` - <a href="#dup" style="color:#856024">делает какую-то копию</a>
  * `dup2` - <a href="#dup2" style="color:#856024">делает копию туда, куда вы хотите</a>
  * `dup3` - <a href="#dup3" style="color:#856024">делает купию туда, куда вы хотите + c указанными опциями</a>
* `pipe` - позволяет создать трубу(=pipe) - получить пару файловых дескрипторов. В один из них можно что-то писать, при этом оно будет становиться доступным для чтения из другого дескриптора. Можно рассматривать pipe как своеобразный файл-очередь.
  * `pipe` - <a href="#pipe" style="color:#856024">делает трубу</a>
  * `pipe2` - <a href="#pipe2" style="color:#856024">делает трубу c указанными опциями</a>
* `fcntl` - универсальная функция, которая умеет делать с открытыми файлами практически все (TODO):
  * <a href="#fcntl_fd_flags" style="color:#856024">Вытащить / установить флаги файлового дескриптора (узнать, открыт ли дескриптор на запись; выставлен ли O_CLOEXEC)</a>
  * <a href="#fcntl_dup" style="color:#856024">Исползовать вместо dup/dup2.</a>
  * Установить размер буффера для каналов.
  * Различные вещи для совместного доступа к файлам.
  * ...

<a href="#hw" style="color:#856024">Комментарии к ДЗ</a>

[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/fdup-pipe)

# dup2

Возможно кто-то из вас видел вызов freopen. Вот это примерно о том же.

In [3]:
%%cpp dup2.cpp
%run gcc dup2.cpp -o dup2.exe
%run ./dup2.exe
%run echo "After program finish" && cat out.txt

#include <stdio.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/types.h>


int main() {
    int fd = open("out.txt", O_WRONLY | O_CREAT | O_TRUNC, 0664);
    dup2(fd, 1); // redirect stdout to file
    close(fd);
    printf("Redirectred 'Hello world!'");
    return 0;
}

Run: `gcc dup2.cpp -o dup2.exe`

Run: `./dup2.exe`

Run: `echo "After program finish" && cat out.txt`

After program finish
Redirectred 'Hello world!'

# dup

In [4]:
%%cpp dup.cpp
%run gcc dup.cpp -o dup.exe
%run ./dup.exe

#include <stdio.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/types.h>

int main() {
    int fd = dup(1);  // копируем stdout в другой дескриптор (значение дескриптора выбирается автоматически)
    dprintf(1, "Write to 1 fd.\n");
    dprintf(fd, "Write to %d fd.\n", fd);
    close(1);
    dprintf(fd, "Write to %d fd after closing 1 fd. (still to stdout)\n", fd);
    close(fd);
    return 0;
}

Run: `gcc dup.cpp -o dup.exe`

Run: `./dup.exe`

Write to 1 fd.
Write to 3 fd.
Write to 3 fd after closing 1 fd. (still to stdout)


## <a name="pipe"></a> pipe и dup2


In [5]:
%%cpp pipe.cpp
%run gcc pipe.cpp -o pipe.exe
%run ./pipe.exe

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/resource.h>
#include <sys/types.h>
#include <sys/wait.h>

int main() {
    int fd[2];
    pipe(fd); // fd[0] - in, fd[1] - out (like stdin=0, stdout=1)
    pid_t pid_1, pid_2;
    if ((pid_1 = fork()) == 0) {
        dup2(fd[1], 1);
        close(fd[0]); // notice: close file descriptors explicitly
        close(fd[1]); // try to comment out and compare behaviour
        // Даже без закрытия сможет отработать (но не надо так делать, лучше всегда закрывать руками)
        execlp("ps", "ps", "aux", NULL);
        assert(0 && "Unreachable position in code if execlp succeeded");
    }
    if ((pid_2 = fork()) == 0) {
        dup2(fd[0], 0);
        close(fd[0]); // notice: close file descriptors explicitly
        close(fd[1]); // try to comment out and compare behaviour
        // ^^^ tail не завершиться пока открыт файловый дескриптор на запись в pipe (он будет ждать данных, которые он бы смог прочитать)
        execlp("tail", "tail", "-n", "4", NULL);
        assert(0 && "Unreachable position in code if execlp succeeded");
    }
    close(fd[0]); // Тут закрыли pipe, потому что он нам больше не нужен (и потому что, если не закроем, то будет ошибка как с программой tail)
    close(fd[1]);
    int status;
    assert(waitpid(pid_1, &status, 0) != -1); // [1.]
    assert(waitpid(pid_2, &status, 0) != -1);
    return 0;
}

Run: `gcc pipe.cpp -o pipe.exe`

Run: `./pipe.exe`

pechatn+    3740  0.0  0.0   2608   604 pts/2    Ss+  21:12   0:00 /usr/bin/sh -c ./pipe.exe
pechatn+    3741  0.0  0.0   2356   576 pts/2    S+   21:12   0:00 ./pipe.exe
pechatn+    3742  0.0  0.1  20124  3284 pts/2    R+   21:12   0:00 ps aux
pechatn+    3743  0.0  0.0  16748   520 pts/2    S+   21:12   0:00 tail -n 4


1. Вопрос: почему нужно делать waitpid после close?
  <br> Ответ: потому что у нас остаётся открытый файловый дескриптор. Дочерний процесс не завершится до тех пор, пока не закроется пайп на запись. А пайп на запись не закроется, пока не закроются все соответствующие файловые дескрипторы (и не только в том же самом процессе). Соответственно, если не сделать close до waitpid, то он просто зависнет. 

# <a name="pipe2"></a> pipe2 и dup2

In [7]:
%%cpp pipe2.cpp
%run gcc pipe2.cpp -o pipe2.exe
%run ./pipe2.exe

#ifndef _GNU_SOURCE
  #define _GNU_SOURCE
#endif
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/resource.h>
#include <sys/types.h>
#include <sys/wait.h>

int main() {
    int fd[2];
    pipe2(fd, O_CLOEXEC); // O_CLOEXEC - created file descriptors will be closed on exec call
    pid_t pid_1, pid_2;
    if ((pid_1 = fork()) == 0) {
        dup2(fd[1], 1); // dup2 doesn't copy O_CLOEXEC attribute
        //dup3(fd[1], 1, O_CLOEXEC); // can compare with this
        execlp("ps", "ps", "aux", NULL);
        assert(0 && "Unreachable position in code if execlp succeeded");
    }
    if ((pid_2 = fork()) == 0) {
        // no close calls here
        dup2(fd[0], 0);
        execlp("tail", "tail", "-n", "4", NULL);
        assert(0 && "Unreachable position in code if execlp succeeded");
    }
    close(fd[0]);
    close(fd[1]);
    int status;
    assert(waitpid(pid_1, &status, 0) != -1);
    assert(waitpid(pid_2, &status, 0) != -1);
    return 0;
}

Run: `gcc pipe2.cpp -o pipe2.exe`

Run: `./pipe2.exe`

pechatn+    3764  0.0  0.0   2608   600 pts/2    Ss+  21:13   0:00 /usr/bin/sh -c ./pipe2.exe
pechatn+    3765  0.0  0.0   2356   584 pts/2    S+   21:13   0:00 ./pipe2.exe
pechatn+    3766  0.0  0.1  20124  3324 pts/2    R+   21:13   0:00 ps aux
pechatn+    3767  0.0  0.0  16748   588 pts/2    S+   21:13   0:00 tail -n 4


Этот пример работал в том числе благодаря тому, что dup2 не копирует O_CLOEXEC флаг. 
Если же вам по каким-то причинам будет нужно, чтобы он копировался, 
то можно исползовать `dup3(fd[1], 1, O_CLOEXEC)`. <a name="dup3"></a>

# <a name="fcntl_dup"></a> fcntl: вместо dup

Варианты использования fcntl для получения и изменения флагов файловых дескрипторов разбирали в sem11.

In [9]:
%%cpp fcntl_dup.cpp
%run gcc fcntl_dup.cpp -o fcntl_dup.exe
%run ./fcntl_dup.exe
%run echo "After program finish" && cat out.txt
%run diff dup2.cpp fcntl_dup.cpp  | grep -v "// %" | grep -e '>' -e '<' -C 1

#include <stdio.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/types.h>


int main() {
    int fd = open("out.txt", O_WRONLY | O_CREAT | O_TRUNC, 0664);
    close(1); // important (в следующей функции просим сделать дескриптор >= 1, поэтому тут нужно закрыть дескриптор 1, чтобы он стал доступен)
    int fd_copy = fcntl(fd, F_DUPFD, 1);
    assert(fd_copy == 1);
    // Три строчки сверху выполняют то же что и dup2(fd, 1)
    // Но не совсем, подумайте, почему не могут быть эквивалентны?
    close(fd);
    printf("Redirectred 'Hello world!'");
    return 0;
}

Run: `gcc fcntl_dup.cpp -o fcntl_dup.exe`

Run: `./fcntl_dup.exe`

Run: `echo "After program finish" && cat out.txt`

After program finish
Redirectred 'Hello world!'

Run: `diff dup2.cpp fcntl_dup.cpp  | grep -v "// %" | grep -e '>' -e '<' -C 1`

15c16,20
<     dup2(fd, 1); // redirect stdout to file
---
>     close(1); // important (в следующей функции просим сделать дескриптор >= 1, поэтому тут нужно закрыть дескриптор 1, чтобы он стал доступен)
>     int fd_copy = fcntl(fd, F_DUPFD, 1);
>     assert(fd_copy == 1);
>     // Три строчки сверху выполняют то же что и dup2(fd, 1)
>     // Но не совсем, подумайте, почему не могут быть эквивалентны?


# fcntl: O_NONBLOCK

In [12]:
%%cpp pipe2.cpp
%run gcc pipe2.cpp -o pipe2.exe
%run ./pipe2.exe

#ifndef _GNU_SOURCE
  #define _GNU_SOURCE
#endif
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/resource.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sched.h>
#include <time.h>
#include <errno.h>

int main() {
    int fd[2];
    pipe(fd); 
    pid_t pid_1, pid_2;
    if ((pid_1 = fork()) == 0) {
        dup2(fd[1], 1); 
        close(fd[0]); 
        close(fd[1]);
        for (int i = 0; i < 1000; ++i) {
            write(1, "X", 1);
            //sched_yield();
            struct timespec t = {.tv_sec = 0, .tv_nsec = 10000};
            nanosleep(&t, &t);  
        }
        return 0;
    }
    if ((pid_2 = fork()) == 0) {
        // no close calls here
        dup2(fd[0], 0);
        close(fd[0]); 
        close(fd[1]);
        fcntl(0, F_SETFL, fcntl(0, F_GETFL, 0) | O_NONBLOCK);
        while (true) {
            char c;
            int r = read(0, &c, 1);
            if (r > 0) {
                write(1, &c, 1);
            } else if (r < 0) {
                assert(errno == EAGAIN);
                write(1, "?", 1);
            } else {
                break;
            }
        }
        return 0;
    }
    close(fd[0]);
    close(fd[1]);
    int status;
    assert(waitpid(pid_1, &status, 0) != -1);
    assert(waitpid(pid_2, &status, 0) != -1);
    return 0;
}

Run: `gcc pipe2.cpp -o pipe2.exe`

Run: `./pipe2.exe`

XX?????????????????????????????????????????????????????????????????????????????????????????????????????????????X?????????????????????????????????????????????????????????????????????????????????????????X??????????????????????????????????X??????????????????X??????????????????????????????????????????????????????X????????????X?????????????????????????????X????????????????????????????????????????????X??????????????????????????????????????????????X?????????????????????????????????????????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????????????????X?????????????????????????????????????????????????X?????????????????????????????????????????????????????????X????????????????????X????????????????X?????????????????????????????????????????????????????????X????????????????????????????????????????????????????????????????X?????????????????X???????????????????????????????????????????????????????X??????????????????????????????????????????

?X?????????????????????????????????????????????X?????????????X???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????X?????????????????????????????????????????????????????????????????????????????????????????????????????X???????????????????????????????????????????????????????X?????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????????X?????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

?????????????????????????????????????????????????X????????????????????????????????????????????????????????????????????????????????X????????????????????????????????????????????X???????????????????????????????????????????????????????X???????????????????????????????????????????????????X??????????????????????????????????????????????X???????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????X???????????????????????????????????????????????????X???????????????????????????????????????????????X?????????????????????????????????????????????????????X???????????????????????????????????????????

???X????????????????????????????????????????????????????????????????X????????X?????????????????????????????????????????????X????????????????????????????????????????????X???????????????????????X?????????????????????????????????????????????????????????????????????X???????X??????????????????????????????X??????????????????????????????????????????????????X?X?????????????????????????????????????????????????????????????????????????X???????????????????????????????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????X?????X???????????????????????????????????????????????????????????????????????????X???????????????????????????????????????????????????????????????????????X?????????????????????????????????????????????????X?X????????????????????????????????????????????????????????????????????????X???????????????????????????????X??????????????????????????????????????????????????????????????????????X??????????????????????????????????????????

?????????????????????????X??????????????????????????????????????????????????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????X???????????????????????????????????????????????????????????????????X????????????????????????????????????????????????????????????????X???????????????????????????????X?????????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????????????????X??????????????????????X?????????????????????????????????????????????????????????????X?????????????????????????????????????????????????????????????????????????X?????????X??????????????????????????????????????????????????????????????????????X??????????????????????????????????????????????????????????????X????????????????????????????????????????????????????????????????????????X?????????????X?????????????????????????????????????????????????????????????????????????X????????X?????????????????????????????????????

# pipe: Эксперимент с вмесимостью

In [21]:
%%cpp pipe_buff.cpp
%run gcc pipe_buff.cpp -o pipe_buff.exe
%run timeout 1 ./pipe_buff.exe 1000 && echo SUCCESS || echo FAILED
%run timeout 1 ./pipe_buff.exe 10000 && echo SUCCESS || echo FAILED
%run timeout 1 ./pipe_buff.exe 100000 && echo SUCCESS || echo FAILED

#ifndef _GNU_SOURCE
  #define _GNU_SOURCE
#endif
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <fcntl.h>
#include <sys/resource.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sched.h>
#include <time.h>
#include <errno.h>

int main(int argc, char** argv) {
    assert(argc == 2);
    int size = strtol(argv[1], NULL, 10);
    int fd[2];
    pipe(fd); 
    char* data = (char*)calloc(size, sizeof(char));
    
    assert(write(fd[1], data, size) == size);
    assert(read(fd[0], data, size) == size);
    
    free(data);
    
    close(fd[0]);
    close(fd[1]);
    return 0;
}

Run: `gcc pipe_buff.cpp -o pipe_buff.exe`

Run: `timeout 1 ./pipe_buff.exe 1000 && echo SUCCESS || echo FAILED`

SUCCESS


Run: `timeout 1 ./pipe_buff.exe 10000 && echo SUCCESS || echo FAILED`

SUCCESS


Run: `timeout 1 ./pipe_buff.exe 100000 && echo SUCCESS || echo FAILED`

FAILED


# pipe: Ошибка записи из-за того, что данные некому читать

In [67]:
%%cpp pipe_err.c
%run gcc pipe_err.c -o pipe_err.exe
%run ./pipe_err.exe ; echo  "Exit code is $?"

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <sys/resource.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sched.h>
#include <time.h>
#include <errno.h>
#include <signal.h>
    
int main() {
    signal(SIGPIPE, SIG_DFL); // форсим дефолтное поведение, так как тут наследуется маска сигналов питона
    
    int size = 100000;
    int fd[2];
    pipe(fd); 

    close(fd[0]);
    char* data = (char*)calloc(size, sizeof(char));
    
    int ret = write(fd[1], data, size);
    if (ret == size) {
        printf("OK\n");
    } else {
        printf("ret = %d\n", ret);
        perror("FAILED to write");
    }
    
    free(data);
    
    int status;
    
    close(fd[1]);
    return 0;
}

Run: `gcc pipe_err.c -o pipe_err.exe`

Run: `./pipe_err.exe ; echo  "Exit code is $?"`

Exit code is 141


141 > 128 -> убит сигналом SIGPIPE = 13 = 141 - 128

То, на что я убил час: если программа запускается питоном, то она игнорит SIGPIPE 

```
[22:33:00 янв 30] pechatnov@pechatnov-vbox:~
  -> (cat ; echo "code $?" 1>&2) | head

code 141
Killed
[22:35:27 янв 30] pechatnov@pechatnov-vbox:~
  -> python3 -c "import os; os.system('(cat ; echo \"code \$?\" 1>&2)')" | head

cat: write error: Broken pipe
code 1
Killed
```

(head килляется вручную и затем пишется перевод строки на вход в обоих случаях)

# <a name="hw"></a> Комментарии к ДЗ

* `inf13-0: posix/pipe/launch` - фактически разобрана, когда переизобретали `freopen`.
* `inf13-1: posix/pipe/connect-2-processes` - тоже фактически разобрана. pipe + работа с дочерними процессами. Для тестирования может быть удобно написать свои программки в виде скриптов с шебангом. [Ссылка для студентов с плохой памятью](https://andreyex.ru/operacionnaya-sistema-linux/shebang-v-bash)
* `inf13-2: posix/pipe/process-gcc-output-2` - как предыдущая, но нужно самим написать программы по разные стороны pipe. Разрешаю использовать `python -c` (типа `python -c 'print "abacaba\nasdfdsv".find("aca")'`)
* `inf13-3: posix/pipe/connect-n-processes` - просто обобщить `inf13-1`
* `inf13-4: posix/pipe/connect-n-processes-one-pipe`
  * В этой задаче важно не забывать про изначально открытые файловые дескрипторы.
  * Когда вы делаете fork и запускается новая программа, она может подгружать динамические библиотеки. И при этом могут создаваться и закрываться файловые дескрипотры. Так что, если у вас открыто 8 файловых дескрипторов, то на fork'е вы скорее всего сломаетесь.
  * Разрешаю хранить промежуточный вывод подпрограмм в памяти основного процесса.
  * В этой задаче можно немного упороться и обойтись двумя пайпами за все время существования программы. Для этого, нужно предполагать, что программы-команды не будут ретраить операцию чтения в случае ошибок. Дальше просто применяем O_NONBLOCK.

**Замечание**

Конечно, однострочники питона это очень плохо в продакшн коде, но очень полезный навык для администрирования всякого-разного.

В качестве простого примера, допустим, что вам нужно красиво отрисовать json и у вас нет для этого встроенных средств:

```bash
[00:44:00 фев 09] pechatnov@pechatnov-osx2:~
  -> echo '{"a": "b", "d": [1, 2, 3]}' | python -c 'import json, sys; json.dump(json.load(sys.stdin), sys.stdout, indent=4)'
{
    "a": "b",
    "d": [
        1,
        2,
        3
    ]
}
```

Тут же можно пофильтровать этот json по какому либо признаку.

В однострочниках есть сложности с циклами и ветвлением, но часто легко обойтись генераторами списков и "тернарными" if'ами.

Так же можно создать файл со скриптом через vim/nano, кто-нибудь из них обычно есть в качестве встроенного редактора.